# Tutorial 04: overview of **compose** stages

The main thing to know about in this section is the `em.KorniaAugmentationPipeline` class, which you can use to add differentiable augmentations to your implanted images. The `kornia` library has dozens of augmentations implemented, so this is a flexible tool for making sure that during training and evaluation you're covering the range of conditions you expect to see in real life.

In [ ]:
import pandas as pd
from PIL import Image
import electricmayhem.whitebox as em

Load a test image...

In [ ]:
#flower = em.load_to_tensor(Image.open("data/flower2.png").resize((128,92)))
flower = em.load_to_tensor("data/flower2.png")
em.plot(flower)

## `em.KorniaAugmentationPipeline`

This is a lightweight wrapper around the `kornia.augmentation` module- define a dictionary of the augmentations you'd like to apply, where each key is the augmentation name and each corresponding value is a dictionary of keywords to initialize it. So if you want to add
```
kornia.augmentation.RandomAffine(degrees=90, shear=10)
```
to your pipeline, initialize `em.KorniaAugmentationPipeline` with
```
em.KorniaAugmentationPipeline({"RandomAffine":{"degrees":90, "shear":10}})
```

In [ ]:
aug_params = {
    "ColorJiggle":{"brightness":0.1, "contrast":0.1, "hue":0.05, "saturation":0.2},
    "RandomAffine":{"degrees":180, "shear":10, "scale":(0.25, 1), "padding_mode":"reflection"},
    "RandomBoxBlur":{},
    "RandomChannelShuffle":{},
    #"RandomGaussianNoise":{"mean":0., "std":0.05},
}
aug = em.KorniaAugmentationPipeline(aug_params)

In [ ]:
augmented, _ = aug(flower.unsqueeze(0))
em.plot(augmented[0])

`kornia` is designed for reproducibility, but I've (infrequently) found some versions of some augmentations that don't reproduce correctly. This matters for the causal analysis computations in `electricmayhem`; if a component in your pipeline isn't reproducible then any "delta" metrics reported will be wrong (since they rely on taking the difference between otherwise-identical batches with and without the patch).

The `em.KorniaAugmentationPipeline.validate()` method will check to make sure identical augmentations will be created during your control steps, and return `True` if your augmentation choices reproduce correctly. The commented-out `RandomGaussianNoise` line above, for example, breaks reproducibility (at least in `kornia==0.7.0`) and should get flagged by the `validate()` method.

In [ ]:
aug.validate(flower.unsqueeze(0))

## `em.PerspectiveTilter`

This stage has a *much* narrower range of application- wrapping `kornia`'s perspective warping tools to make an image look like it's falling back away from the camera (for example, if you wanted to make an image appear to be off-nadir).

In [ ]:
tilt = em.PerspectiveTilter()

In [ ]:
tilted, _ = tilt(flower.unsqueeze(0))
em.plot(tilted[0])